# Acquire data

This notebook loads all of the csv files and saves them to a single pandas dataframe

In [24]:
import pandas as pd
import os
from datetime import datetime as dt

In [25]:
df = pd.DataFrame()

path = '../data/raw/'

for filename in os.listdir('../data/raw'):
    if filename[-4:] == '.csv':
        df = df.append(pd.read_csv(path + filename, encoding='latin1'))

df = df.sort_values('Date (GMT)', ascending=False).drop_duplicates(['GUID', 'URL']).reset_index(drop=True)

In [26]:
df.to_pickle('../data/interim/aggregated.pkl')

In [37]:
df[['Contents', 'Author']].head(20)

,Contents,Author
0,Rudd goes in for a strong and stable photobomb...,@sharmilabousa
1,RT @gareth13479 @AmberRuddHR gets the prize fo...,@deadpubs
2,RT @TasminaSheikh Team @theSNP Auchterarder of...,@IsThisAB0t
3,@LarmySausage @HackneyAbbott @JuliaHB1 @AmberR...,@MsJoNeary
4,RT @BarbPitruz @TulipSiddiq brilliant as alway...,@RoxethMyLord
5,@andrewj2009 no joke no one is here to suppor...,@king_darran
6,RT @HackneyAbbott In tough Brexit negotiations...,@AliMalik1x
7,RT @HackneyAbbott In tough Brexit negotiations...,@Elin012
8,RT @HackneyAbbott .@AmberRuddHR Heard you were...,@CaMrAn_12
9,RT @HackneyAbbott In tough Brexit negotiations...,@AFILLEMON
